In [8]:
import requests
import os
from calendar import monthrange
from multiprocessing import Pool


def add_days(date, days):
    dsamnt = monthrange(date[0], date[1])
    date[2] += days
    if date[2] > dsamnt[1]:
        date[2] -= dsamnt[1]
        date[1] += 1
    if date[1] == 13:
        date[1] = 1
        date[0] += 1
    
downld_sea = "/mnt/e/ArtFiles2/project/Storm and sea level/data/sea/"
downld_wind = "/mnt/e/ArtFiles2/project/Storm and sea level/data/wind/"
#[y, m, d]
start = [2011, 4, 1]
#end = [2015, 1, 1]
end = [2019, 1, 1]
q_amnt = 12

In [7]:
def fetch_url_sea(item):
    cur = item[0]
    cursave = item[1]
    name = item[2]
    login = item[3]
    pwd = item[4]
    with open(cursave, "wb") as f:
        response = requests.get(cur, auth=(login, pwd), stream=True)
        print("Downloading %s Status code %s" % (name, response.status_code))
        for chunk in response:
            f.write(chunk)
    print("%s is Done!" % name)


cdate = list()
cdate.append(start[0])
cdate.append(start[1])
cdate.append(start[2])
basename = "ssh_grids_v1812_"
endname = "12.nc"
url = "https://podaac-tools.jpl.nasa.gov/drive/files/allData/merged_alt/L4/cdr_grid/"
savefile = downld_sea
p = Pool(q_amnt)
queries = list()
while cdate[0] != end[0] or cdate[1] != end[1]:
    if len(queries) == q_amnt:
        p.imap(fetch_url_sea, queries)
        queries = list()
    name = str(cdate[0])
    if len(str(cdate[1])) == 1:
        name += '0'
    name += str(cdate[1])
    if len(str(cdate[2])) == 1:
        name += '0'
    name += str(cdate[2])
    name = basename + name + endname
    cur = url + name
    cursave = savefile + name 
    login = "artgoldman"
    pwd = "XiB@2xdHhSHzNu@vvWr"
    #r = requests.get(cur, auth=(login, pwd))
    response = requests.get(cur, auth=(login, pwd), stream=True)
    if response.status_code != 200:
        print("Downloading %s Status code ERROR %s" % (name, response.status_code))
        add_days(cdate, 1)
    else:
        itm = [cur, cursave, name, login, pwd]
        queries.append(itm)
        add_days(cdate, 5)
        
    
p.imap(fetch_url_sea, queries)  
p.close()
p.join()
print("Download finished!")

ssh_grids_v1812_2015010512.nc is Done!
ssh_grids_v1812_2015021412.nc is Done!
ssh_grids_v1812_2015030112.nc is Done!
ssh_grids_v1812_2015021912.nc is Done!
ssh_grids_v1812_2015013012.nc is Done!
ssh_grids_v1812_2015012512.nc is Done!
ssh_grids_v1812_2015011512.nc is Done!
ssh_grids_v1812_2015012012.nc is Done!
ssh_grids_v1812_2015020412.nc is Done!
ssh_grids_v1812_2015011012.nc is Done!
ssh_grids_v1812_2015020912.nc is Done!
ssh_grids_v1812_2015022412.nc is Done!
ssh_grids_v1812_2015031112.nc is Done!
ssh_grids_v1812_2015031612.nc is Done!
ssh_grids_v1812_2015032112.nc is Done!
ssh_grids_v1812_2015030612.nc is Done!
ssh_grids_v1812_2015042012.nc is Done!
ssh_grids_v1812_2015033112.nc is Done!
ssh_grids_v1812_2015043012.nc is Done!
ssh_grids_v1812_2015041012.nc is Done!
ssh_grids_v1812_2015040512.nc is Done!
ssh_grids_v1812_2015041512.nc is Done!
ssh_grids_v1812_2015032612.nc is Done!
ssh_grids_v1812_2015050512.nc is Done!
ssh_grids_v1812_2015042512.nc is Done!
ssh_grids_v1812_201505301

ssh_grids_v1812_2016010512.nc is Done!
ssh_grids_v1812_2016012512.nc is Done!
ssh_grids_v1812_2015122112.nc is Done!
ssh_grids_v1812_2016012012.nc is Done!
ssh_grids_v1812_2016013012.nc is Done!
ssh_grids_v1812_2016011012.nc is Done!
ssh_grids_v1812_2016020412.nc is Done!
ssh_grids_v1812_2016020912.nc is Done!
ssh_grids_v1812_2016021412.nc is Done!
ssh_grids_v1812_2016011512.nc is Done!
ssh_grids_v1812_2016021912.nc is Done!
ssh_grids_v1812_2016032012.nc is Done!
ssh_grids_v1812_2016030512.nc is Done!
ssh_grids_v1812_2016022912.nc is Done!
ssh_grids_v1812_2016022412.nc is Done!
ssh_grids_v1812_2016033012.nc is Done!
ssh_grids_v1812_2016031012.nc is Done!
ssh_grids_v1812_2016031512.nc is Done!
ssh_grids_v1812_2016040912.nc is Done!
ssh_grids_v1812_2016032512.nc is Done!
ssh_grids_v1812_2016042912.nc is Done!
ssh_grids_v1812_2016041412.nc is Done!
ssh_grids_v1812_2016041912.nc is Done!
ssh_grids_v1812_2016040412.nc is Done!
ssh_grids_v1812_2016051912.nc is Done!
ssh_grids_v1812_201605091

ssh_grids_v1812_2017021312.nc is Done!
ssh_grids_v1812_2017032512.nc is Done!
ssh_grids_v1812_2017032012.nc is Done!
ssh_grids_v1812_2017031512.nc is Done!
ssh_grids_v1812_2017022812.nc is Done!
ssh_grids_v1812_2017040412.nc is Done!
ssh_grids_v1812_2017040912.nc is Done!
ssh_grids_v1812_2017033012.nc is Done!
ssh_grids_v1812_2017042412.nc is Done!
ssh_grids_v1812_2017052412.nc is Done!
ssh_grids_v1812_2017041912.nc is Done!
ssh_grids_v1812_2017041412.nc is Done!
ssh_grids_v1812_2017051912.nc is Done!
ssh_grids_v1812_2017050412.nc is Done!
ssh_grids_v1812_2017050912.nc is Done!
ssh_grids_v1812_2017042912.nc is Done!
ssh_grids_v1812_2017060312.nc is Done!
ssh_grids_v1812_2017051412.nc is Done!
ssh_grids_v1812_2017052912.nc is Done!
ssh_grids_v1812_2017061312.nc is Done!
ssh_grids_v1812_2017060812.nc is Done!
ssh_grids_v1812_2017062812.nc is Done!
ssh_grids_v1812_2017072312.nc is Done!
ssh_grids_v1812_2017062312.nc is Done!
ssh_grids_v1812_2017070312.nc is Done!
ssh_grids_v1812_201707081

ssh_grids_v1812_2018042912.nc is Done!
ssh_grids_v1812_2018041912.nc is Done!
ssh_grids_v1812_2018051412.nc is Done!
ssh_grids_v1812_2018050912.nc is Done!
ssh_grids_v1812_2018052412.nc is Done!
ssh_grids_v1812_2018051912.nc is Done!
ssh_grids_v1812_2018060312.nc is Done!
ssh_grids_v1812_2018052912.nc is Done!
ssh_grids_v1812_2018060812.nc is Done!
ssh_grids_v1812_2018072312.nc is Done!
ssh_grids_v1812_2018062812.nc is Done!
ssh_grids_v1812_2018061312.nc is Done!
ssh_grids_v1812_2018061812.nc is Done!
ssh_grids_v1812_2018062312.nc is Done!
ssh_grids_v1812_2018080712.nc is Done!
ssh_grids_v1812_2018070312.nc is Done!
ssh_grids_v1812_2018071312.nc is Done!
ssh_grids_v1812_2018072812.nc is Done!
ssh_grids_v1812_2018080212.nc is Done!
ssh_grids_v1812_2018071812.nc is Done!
ssh_grids_v1812_2018070812.nc is Done!
ssh_grids_v1812_2018081212.nc is Done!
ssh_grids_v1812_2018082712.nc is Done!
ssh_grids_v1812_2018082212.nc is Done!
ssh_grids_v1812_2018090612.nc is Done!
ssh_grids_v1812_201809111

In [10]:
def fetch_url_wind(item):
    cur = item[0]
    cursave = item[1]
    name = item[2]
    with open(cursave, "wb") as f:
        response = requests.get(cur, stream=True)
        if response.status_code != 200:
            print("Downloading %s Status code ERROR %s" % (name, response.status_code))
        else:
            print("Downloading %s Status code %s" % (name, response.status_code))
            for chunk in response:
                f.write(chunk)
    print("%s is Done!" % name)

cdate = list()
cdate.append(start[0])
cdate.append(start[1])
cdate.append(start[2])
basename = "wnd10m.l.gdas."
#endname = ".grb2"
endname = ".grib2"
#url = "https://nomads.ncdc.noaa.gov/data/cfsr/"
url = "https://nomads.ncdc.noaa.gov/modeldata/cfsv2_analysis_timeseries/"
savefile = downld_wind
p = Pool(q_amnt)
while cdate[0] != end[0] or cdate[1] != end[1]:
    name = str(cdate[0])
    if len(str(cdate[1])) == 1:
        name += '0'
    name += str(cdate[1])
    cur = url + str(cdate[0])+"/"+name +"/" + basename + name + endname
    cursave = savefile + basename + name + endname
    itm = [cur, cursave, basename + name + endname]
    p.imap(fetch_url_wind, [itm])
    #p.map(fetch_url_wind, itm)
    cdate[1] += 1
    if cdate[1] == 13:
        cdate[1] = 1
        cdate[0] += 1
p.close()
p.join()
print("Download finished!")

wnd10m.l.gdas.201005.grib2 is Done!
wnd10m.l.gdas.201003.grib2 is Done!
wnd10m.l.gdas.201001.grib2 is Done!
wnd10m.l.gdas.201008.grib2 is Done!
wnd10m.l.gdas.201007.grib2 is Done!
wnd10m.l.gdas.201002.grib2 is Done!
wnd10m.l.gdas.201010.grib2 is Done!
wnd10m.l.gdas.201012.grib2 is Done!
wnd10m.l.gdas.201006.grib2 is Done!
wnd10m.l.gdas.201004.grib2 is Done!
wnd10m.l.gdas.201011.grib2 is Done!
wnd10m.l.gdas.201009.grib2 is Done!
wnd10m.l.gdas.201101.grib2 is Done!
wnd10m.l.gdas.201103.grib2 is Done!
wnd10m.l.gdas.201102.grib2 is Done!
wnd10m.l.gdas.201112.grib2 is Done!
wnd10m.l.gdas.201204.grib2 is Done!
wnd10m.l.gdas.201109.grib2 is Done!
wnd10m.l.gdas.201105.grib2 is Done!
wnd10m.l.gdas.201111.grib2 is Done!
wnd10m.l.gdas.201104.grib2 is Done!
wnd10m.l.gdas.201205.grib2 is Done!
wnd10m.l.gdas.201106.grib2 is Done!
wnd10m.l.gdas.201108.grib2 is Done!
wnd10m.l.gdas.201107.grib2 is Done!
wnd10m.l.gdas.201203.grib2 is Done!
wnd10m.l.gdas.201208.grib2 is Done!
wnd10m.l.gdas.201207.grib2 i

wnd10m.l.gdas.201701.grib2 is Done!
wnd10m.l.gdas.201703.grib2 is Done!
wnd10m.l.gdas.201609.grib2 is Done!
wnd10m.l.gdas.201704.grib2 is Done!
wnd10m.l.gdas.201705.grib2 is Done!
wnd10m.l.gdas.201706.grib2 is Done!
wnd10m.l.gdas.201708.grib2 is Done!
wnd10m.l.gdas.201707.grib2 is Done!
wnd10m.l.gdas.201702.grib2 is Done!
wnd10m.l.gdas.201709.grib2 is Done!
wnd10m.l.gdas.201711.grib2 is Done!
wnd10m.l.gdas.201802.grib2 is Done!
wnd10m.l.gdas.201803.grib2 is Done!
wnd10m.l.gdas.201710.grib2 is Done!
wnd10m.l.gdas.201805.grib2 is Done!
wnd10m.l.gdas.201801.grib2 is Done!
wnd10m.l.gdas.201804.grib2 is Done!
wnd10m.l.gdas.201806.grib2 is Done!
wnd10m.l.gdas.201712.grib2 is Done!
wnd10m.l.gdas.201812.grib2 is Done!
wnd10m.l.gdas.201809.grib2 is Done!
wnd10m.l.gdas.201807.grib2 is Done!
wnd10m.l.gdas.201808.grib2 is Done!
wnd10m.l.gdas.201811.grib2 is Done!
wnd10m.l.gdas.201810.grib2 is Done!
Download finished!
